In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163755 sha256=af8b5c1f8d882005e573de0bf8f29865fb8ff08699da64a03de8c9c135cc3ac7
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df0 = pd.read_csv("/content/drive/MyDrive/ml-1m/df_cat0.csv")
df1 = pd.read_csv("/content/drive/MyDrive/ml-1m/df_cat1.csv")
df2 = pd.read_csv("/content/drive/MyDrive/ml-1m/df_cat2.csv")

column_names = ['User_ID', 'MovieID', 'Rating', 'Timestamp']
ratings = pd.read_csv("/content/drive/MyDrive/ml-1m/ratings.dat", sep = "::", names = column_names, engine='python')

column_names = ['User_ID', 'Gender', 'Age', 'Occupation', 'Zip-code']
users = pd.read_csv("/content/drive/MyDrive/ml-1m/users.dat", sep = "::", names = column_names, engine='python')

In [ ]:
ratings = ratings.drop(['Timestamp'], axis=1)
users = users.drop(['Zip-code'],axis=1)

In [ ]:
def map_gender(value):
    if value == 'M':
        return 0
    return 1
users['Gender'] = users['Gender'].apply(map_gender)

In [ ]:
# df0.fillna(0, inplace=True)
# df1.fillna(0, inplace=True)
# df2.fillna(0, inplace=True)

# Content Based Recommendation

In [ ]:
# Define a function to get movie recommendations based on movie title
def get_content_based_recommendations(movies, title, cosine_sim, content_n = 10):
    idx = movies.index[movies['Title'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:content_n+1]  # Top content_n similar movies
    movie_indices = [i[0] for i in sim_scores]
    return movies['Title'].iloc[movie_indices]

In [ ]:
df0_movie_ratings = pd.merge(df0, ratings, on='MovieID')

df0_tfidf_vectorizer = TfidfVectorizer()
df0_tfidf_matrix = df0_tfidf_vectorizer.fit_transform(df0['Genre 1'] + ' ' + df0['Genre 2'] + ' ' + df0['Genre 3'] + ' ' + df0['Genre 4'])

df0_cosine_sim = linear_kernel(df0_tfidf_matrix, df0_tfidf_matrix)

In [ ]:
df1_movie_ratings = pd.merge(df1, ratings, on='MovieID')

df1_tfidf_vectorizer = TfidfVectorizer()
df1_tfidf_matrix = df1_tfidf_vectorizer.fit_transform(df1['Genre 1'] + ' ' + df1['Genre 2'] + ' ' + df1['Genre 3'] + ' ' + df1['Genre 4'])

df1_cosine_sim = linear_kernel(df1_tfidf_matrix, df1_tfidf_matrix)

In [ ]:
df2_movie_ratings = pd.merge(df2, ratings, on='MovieID')

df2_tfidf_vectorizer = TfidfVectorizer()
df2_tfidf_matrix = df2_tfidf_vectorizer.fit_transform(df2['Genre 1'] + ' ' + df2['Genre 2'] + ' ' + df2['Genre 3'] + ' ' + df2['Genre 4'])

df2_cosine_sim = linear_kernel(df2_tfidf_matrix, df2_tfidf_matrix)

In [ ]:
user_favorite_movie = 'Sudden Death (1995)'
recommended_movies = get_content_based_recommendations(df0, user_favorite_movie, df0_cosine_sim)
print(recommended_movies)

1                       Money Train (1995)
2                         Fair Game (1995)
3                          Bad Boys (1995)
4     Under Siege 2: Dark Territory (1995)
5                         Drop Zone (1994)
6                       Hunted, The (1995)
7                   Specialist, The (1994)
8                       Bad Company (1995)
9                    Street Fighter (1994)
10                      Coldblooded (1995)
Name: Title, dtype: object


In [ ]:
def calculate_average_rating(movies, ratings, cosine_sim):
    total_rating = 0
    total_movies = 0

    for idx, movie_row in movies.iterrows():
        movie_title = movie_row['Title']
        recommended_movies = get_content_based_recommendations(movies, movie_title, cosine_sim)

        for recommended_movie_title in recommended_movies:
            # Find the actual ratings for the recommended movie
            recommended_movie_id = movies[movies['Title'] == recommended_movie_title]['MovieID'].values[0]
            actual_ratings = ratings[ratings['MovieID'] == recommended_movie_id]['Rating']

            if not actual_ratings.empty and not actual_ratings.isna().any():
                # Calculate the average actual rating
                average_actual_rating = actual_ratings.mean()

                # Calculate the error
                total_rating += average_actual_rating
                total_movies += 1
                # print(total_rating)
    # Calculate the average error
    average_rating = total_rating / total_movies if total_movies > 0 else 0

    return average_rating

In [ ]:
df0_content_rating = calculate_average_rating(df0, df0_movie_ratings, df0_cosine_sim)
print("Average recommender rating for category 0:", df0_content_rating)

df1_content_rating = calculate_average_rating(df1, df1_movie_ratings, df1_cosine_sim)
print("Average recommender rating for category 1:", df1_content_rating)

df2_content_rating = calculate_average_rating(df2, df2_movie_ratings, df2_cosine_sim)
print("Average recommender rating for category 2:", df2_content_rating)

Average recommender rating for category 0: 2.7888029720654215
Average recommender rating for category 1: 3.210108934965727
Average recommender rating for category 2: 3.2979804143559197


# Collaborative Filtering Recommendation

In [ ]:
def get_collaborative_recommendations(movies, ratings, user_id, n=10):
    user_ratings = ratings[ratings['User_ID'] == user_id]
    user_unrated_movies = movies[~movies['MovieID'].isin(user_ratings['MovieID'])]
    predictions = []
    for movie_id in user_unrated_movies['MovieID']:
        movie_prediction = model.predict(user_id, movie_id)
        predictions.append((movie_id, movie_prediction.est))
    predictions.sort(key=lambda x: x[1], reverse=True)
    top_predictions = predictions[:n]
    recommended_movie_ids = [x[0] for x in top_predictions]
    return movies[movies['MovieID'].isin(recommended_movie_ids)]['Title']

In [ ]:
# Load the ratings data using Surprise library
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['User_ID', 'MovieID', 'Rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)

# Build a collaborative filtering model (SVD)
model = SVD()
model.fit(trainset)

In [ ]:
# Make predictions on the test set
predictions = model.test(testset)

# Calculate RMSE (Root Mean Square Error) to evaluate the model's performance
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

RMSE: 0.8747
RMSE: 0.8747180696383404


In [ ]:
mae = accuracy.mae(predictions)
print(f"MAE: {mae}")

MAE:  0.6858
MAE: 0.6857718575931245


In [ ]:
# Example usage
user_id = 1
recommended_movies = get_collaborative_recommendations(df2, ratings,user_id)
print(recommended_movies)

144            Shawshank Redemption, The (1994)
452                It Happened One Night (1934)
470                It's a Wonderful Life (1946)
472                     Bringing Up Baby (1938)
823                    Good Will Hunting (1997)
1170                         October Sky (1999)
1400    Conformist, The (Il Conformista) (1970)
1460                Face in the Crowd, A (1957)
1650                     For All Mankind (1989)
1709                    Inherit the Wind (1960)
Name: Title, dtype: object


In [ ]:
def calculate_average_rating_collab():
    df0_average_rating = 0
    df0_cnt = 0

    df1_average_rating = 0
    df1_cnt = 0

    df2_average_rating = 0
    df2_cnt = 0

    for user_id in users['User_ID']:
        df0_recommended_movies = get_collaborative_recommendations(df0, ratings, user_id)
        df1_recommended_movies = get_collaborative_recommendations(df1, ratings, user_id)
        df2_recommended_movies = get_collaborative_recommendations(df2, ratings, user_id)

        total_rating_df0 = 0
        total_movies_df0 = 0

        for recommended_movie_title in df0_recommended_movies:
            # Find the actual ratings for the recommended movie in df0_movie_ratings
            recommended_movie_id = df0[df0['Title'] == recommended_movie_title]['MovieID'].values[0]
            actual_ratings = df0_movie_ratings[df0_movie_ratings['MovieID'] == recommended_movie_id]['Rating']

            if not actual_ratings.empty and not actual_ratings.isna().any():
                # Calculate the average actual rating
                mean_rating = actual_ratings.mean()

                # Accumulate the rating
                total_rating_df0 += mean_rating
                total_movies_df0 += 1

        if total_movies_df0 > 0:
            average_rating_df0 = total_rating_df0 / total_movies_df0
            df0_average_rating += average_rating_df0  # Accumulate the rating
            df0_cnt += 1

        total_rating_df1 = 0
        total_movies_df1 = 0

        for recommended_movie_title in df1_recommended_movies:
            # Find the actual ratings for the recommended movie in df1_movie_ratings
            recommended_movie_id = df1[df1['Title'] == recommended_movie_title]['MovieID'].values[0]
            actual_ratings = df1_movie_ratings[df1_movie_ratings['MovieID'] == recommended_movie_id]['Rating']

            if not actual_ratings.empty and not actual_ratings.isna().any():
                # Calculate the average actual rating
                mean_rating = actual_ratings.mean()

                # Accumulate the rating
                total_rating_df1 += mean_rating
                total_movies_df1 += 1

        if total_movies_df1 > 0:
            average_rating_df1 = total_rating_df1 / total_movies_df1
            df1_average_rating += average_rating_df1  # Accumulate the rating
            df1_cnt += 1

        total_rating_df2 = 0
        total_movies_df2 = 0

        for recommended_movie_title in df2_recommended_movies:
            # Find the actual ratings for the recommended movie in df2_movie_ratings
            recommended_movie_id = df2[df2['Title'] == recommended_movie_title]['MovieID'].values[0]
            actual_ratings = df2_movie_ratings[df2_movie_ratings['MovieID'] == recommended_movie_id]['Rating']

            if not actual_ratings.empty and not actual_ratings.isna().any():
                # Calculate the average actual rating
                mean_rating = actual_ratings.mean()

                # Accumulate the rating
                total_rating_df2 += mean_rating
                total_movies_df2 += 1

        if total_movies_df2 > 0:
            average_rating_df2 = total_rating_df2 / total_movies_df2
            df2_average_rating += average_rating_df2  # Accumulate the rating
            df2_cnt += 1

    # Calculate and return the average ratings for each category
    return df0_average_rating / df0_cnt, df1_average_rating / df1_cnt, df2_average_rating / df2_cnt

df0_collab_rating, df1_collab_rating, df2_collab_rating = calculate_average_rating_collab()

In [ ]:
print("Average movie rating of collaborative filtering for category 0:", df0_collab_rating)
print("Average movie rating of collaborative filtering for category 1:", df1_collab_rating)
print("Average movie rating of collaborative filtering for category 2:", df2_collab_rating)

Average movie rating of collaborative filtering for category 0: 3.6848266629942468
Average movie rating of collaborative filtering for category 1: 4.3311364233972505
Average movie rating of collaborative filtering for category 2: 4.255159457857862


# Hybrid Recommendation system

In [ ]:
def determine_category(user_favorite_movie, df0, df1, df2):

    # Define the MovieID ranges for each category
    df0_movie_ids = df0['Title'].unique()
    df1_movie_ids = df1['Title'].unique()
    df2_movie_ids = df2['Title'].unique()

    # Check which category the user's favorite movie falls into
    if user_favorite_movie in df0_movie_ids:
        return 'df0'
    elif user_favorite_movie in df1_movie_ids:
        return 'df1'
    elif user_favorite_movie in df2_movie_ids:
        return 'df2'
    else:
        return 'unknown'

In [ ]:
def get_hybrid_recommendations(user_id, user_favorite_movie, content_n=5, collaborative_n=5):

    user_favorite_movie_category = determine_category(user_favorite_movie, df0, df1, df2)

    # Based on the category, use the corresponding cosine similarity matrix
    if user_favorite_movie_category == 'df0':
        movies = df0
        cosine_sim = df0_cosine_sim
        df_ratings = df0_movie_ratings
    elif user_favorite_movie_category == 'df1':
        movies = df1
        cosine_sim = df1_cosine_sim
        df_ratings = df1_movie_ratings
    elif user_favorite_movie_category == 'df2':
        movies = df2
        cosine_sim = df2_cosine_sim
        df_ratings = df2_movie_ratings

    # Get content-based and collaborative-based recommendations
    content_recommendations = get_content_based_recommendations(movies, user_favorite_movie, cosine_sim)
    collaborative_recommendations = get_collaborative_recommendations(movies,  df_ratings, user_id, collaborative_n)

    # Find common movies in both sets of recommendations
    common_movies = set(content_recommendations) & set(collaborative_recommendations)

    # Create a list of ranked recommendations
    ranked_recommendations = list(common_movies) + list(set(content_recommendations) - common_movies) + list(set(collaborative_recommendations) - common_movies)

    # Return the top content_n + collaborative_n recommendations
    return ranked_recommendations[:content_n + collaborative_n]

# Example usage
user_id = 1  # Replace with the user ID for which you want recommendations
user_favorite_movie = 'Father of the Bride Part II (1995)'
recommended_movies = get_hybrid_recommendations(user_id, user_favorite_movie, content_n=5, collaborative_n=5)
print(recommended_movies)

['Mr. Wrong (1996)', "Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996)", 'Mighty Aphrodite (1995)', 'Ace Ventura: When Nature Calls (1995)', 'In the Bleak Midwinter (1995)', 'Friday (1995)', 'Black Sheep (1996)', 'Bottle Rocket (1996)', 'It Takes Two (1995)', 'Bio-Dome (1996)']


In [ ]:
def calculate_average_rating_hybrid():
    df0_average_rating = 0
    df0_cnt = 0

    df1_average_rating = 0
    df1_cnt = 0

    df2_average_rating = 0
    df2_cnt = 0

    for user_id in users['User_ID']:
        for movie in df0['Title']:
            # Get content-based recommendations for the user and movie
            content_recommendations = get_content_based_recommendations(df0, movie, df0_cosine_sim)

            # Get collaborative-based recommendations for the user
            collaborative_recommendations = get_collaborative_recommendations(df0, df0_movie_ratings, user_id)

            # Find common movies in both sets of recommendations
            common_movies = set(content_recommendations) & set(collaborative_recommendations)

            total_rating_df0 = 0
            total_movies_df0 = 0

            for recommended_movie_title in common_movies:
                # Exclude the user's favorite movie from recommendations
                if recommended_movie_title != movie:
                    # Find the actual ratings for the recommended movie
                    recommended_movie_id = df0[df0['Title'] == recommended_movie_title]['MovieID'].values[0]
                    actual_ratings = df0_movie_ratings[df0_movie_ratings['MovieID'] == recommended_movie_id]['Rating']

                    if not actual_ratings.empty and not actual_ratings.isna().any():
                        # Calculate the average actual rating
                        mean_rating = actual_ratings.mean()
                        total_rating_df0 += mean_rating
                        total_movies_df0 += 1

            if total_movies_df0 > 0:
                average_rating_df0 = total_rating_df0 / total_movies_df0
                df0_average_rating += average_rating_df0
                df0_cnt += 1

        for movie in df1['Title']:
            # Get content-based recommendations for the user and movie
            content_recommendations = get_content_based_recommendations(df1,movie, df1_cosine_sim)

            # Get collaborative-based recommendations for the user
            collaborative_recommendations = get_collaborative_recommendations(df1, df1_movie_ratings, user_id)

            # Find common movies in both sets of recommendations
            common_movies = set(content_recommendations) & set(collaborative_recommendations)

            total_rating_df1 = 0
            total_movies_df1 = 0

            for recommended_movie_title in common_movies:
                # Exclude the user's favorite movie from recommendations
                if recommended_movie_title != movie:
                    # Find the actual ratings for the recommended movie
                    recommended_movie_id = df1[df1['Title'] == recommended_movie_title]['MovieID'].values[0]
                    actual_ratings = df1_movie_ratings[df1_movie_ratings['MovieID'] == recommended_movie_id]['Rating']

                    if not actual_ratings.empty and not actual_ratings.isna().any():
                        # Calculate the average actual rating
                        mean_rating = actual_ratings.mean()
                        total_rating_df1 += mean_rating
                        total_movies_df1 += 1

            if total_movies_df1 > 0:
                average_rating_df1 = total_rating_df1 / total_movies_df1
                df1_average_rating += average_rating_df1
                df1_cnt += 1

        for movie in df2['Title']:
            # Get content-based recommendations for the user and movie
            content_recommendations = get_content_based_recommendations(df2, movie, df2_cosine_sim)

            # Get collaborative-based recommendations for the user
            collaborative_recommendations = get_collaborative_recommendations(df2, df2_movie_ratings, user_id)

            # Find common movies in both sets of recommendations
            common_movies = set(content_recommendations) & set(collaborative_recommendations)

            total_rating_df2 = 0
            total_movies_df2 = 0

            for recommended_movie_title in common_movies:
                # Exclude the user's favorite movie from recommendations
                if recommended_movie_title != movie:
                    # Find the actual ratings for the recommended movie
                    recommended_movie_id = df2[df2['Title'] == recommended_movie_title]['MovieID'].values[0]
                    actual_ratings = df2_movie_ratings[df2_movie_ratings['MovieID'] == recommended_movie_id]['Rating']

                    if not actual_ratings.empty and not actual_ratings.isna().any():
                        # Calculate the average actual rating
                        mean_rating = actual_ratings.mean()
                        total_rating_df2 += mean_rating
                        total_movies_df2 += 1

            if total_movies_df2 > 0:
                average_rating_df2 = total_rating_df2 / total_movies_df2
                df2_average_rating += average_rating_df2
                df2_cnt += 1

    return df0_average_rating / df0_cnt, df1_average_rating / df1_cnt, df2_average_rating / df2_cnt
df0_hybrid_rating , df1_hybrid_rating , df2_hybrid_rating  = calculate_average_rating_hybrid()
print('Hybrid recommender syste. average rating for category 0:', df0_hybrid_rating)
print('Hybrid recommender syste. average rating for category 1:', df1_hybrid_rating)
print('Hybrid recommender syste. average rating for category 2:', df2_hybrid_rating)

Hybrid recommender syste. average rating for category 0: 3.524485691318226
Hybrid recommender syste. average rating for category 1: 4.359161635313443
Hybrid recommender syste. average rating for category 2: 4.227366515440555


In [ ]:
print('Hybrid recommender syste. average rating for category 0:', df0_hybrid_rating)
print('Hybrid recommender syste. average rating for category 1:', df1_hybrid_rating)
print('Hybrid recommender syste. average rating for category 2:', df2_hybrid_rating)

Hybrid recommender syste. average rating for category 0: 3.524485691318226
Hybrid recommender syste. average rating for category 1: 4.359161635313443
Hybrid recommender syste. average rating for category 2: 4.227366515440555


In [ ]:
content_based_weight_0 =  df0_content_rating
content_based_weight_1 =  df1_content_rating
content_based_weight_2 =  df2_content_rating

collaborative_based_weight_0 = df0_collab_rating
collaborative_based_weight_1 = df1_collab_rating
collaborative_based_weight_2 = df2_collab_rating

hybrid_model_weight_0 = df0_hybrid_rating
hybrid_model_weight_1 = df1_hybrid_rating
hybrid_model_weight_2 = df2_hybrid_rating

In [ ]:
import pandas as pd

def get_recommendations(user_id, title):
    user_favorite_movie_category = determine_category(title, df0, df1, df2)  # Implement your logic here

    movies = None
    cosine_sim = None
    df_ratings = None

    # Based on the category, use the corresponding cosine similarity matrix and weights
    if user_favorite_movie_category == 'df0':
        movies = df0
        cosine_sim = df0_cosine_sim
        df_ratings = df0_movie_ratings
        content_based_weight = content_based_weight_0
        collaborative_based_weight = collaborative_based_weight_0
        hybrid_model_weight = hybrid_model_weight_0
    elif user_favorite_movie_category == 'df1':
        movies = df1
        cosine_sim = df1_cosine_sim
        df_ratings = df1_movie_ratings
        content_based_weight = content_based_weight_1
        collaborative_based_weight = collaborative_based_weight_1
        hybrid_model_weight = hybrid_model_weight_1
    elif user_favorite_movie_category == 'df2':
        movies = df2
        cosine_sim = df2_cosine_sim
        df_ratings = df2_movie_ratings
        content_based_weight = content_based_weight_2
        collaborative_based_weight = collaborative_based_weight_2
        hybrid_model_weight = hybrid_model_weight_2

    content_based_recommendations = get_content_based_recommendations(movies, title, cosine_sim)
    collaborative_based_recommendations = get_collaborative_recommendations(movies, ratings, user_id, n=10)
    hybrid_model_recommendations = get_hybrid_recommendations(user_id, content_n=10, collaborative_n=10)

    # Merge recommendation lists with df_ratings to get the corresponding ratings
    content_based_ratings = df_ratings[df_ratings['Title'].isin(content_based_recommendations)]
    collaborative_based_ratings = df_ratings[df_ratings['Title'].isin(collaborative_based_recommendations)]
    hybrid_model_ratings = df_ratings[df_ratings['Title'].isin(hybrid_model_recommendations)]

    # Apply weights to the ratings using .loc to avoid the SettingWithCopyWarning
    content_based_ratings.loc[:, 'Rating'] *= content_based_weight/5
    collaborative_based_ratings.loc[:, 'Rating'] *= collaborative_based_weight/5
    hybrid_model_ratings.loc[:, 'Rating'] *= hybrid_model_weight/5

    # Create dataframes for each recommendation method
    content_based_df = content_based_ratings[['Title', 'Rating']]
    collaborative_based_df = collaborative_based_ratings[['Title', 'Rating']]
    hybrid_model_df = hybrid_model_ratings[['Title', 'Rating']]

    # Concatenate all three dataframes into a single dataframe
    all_recommendations_df = pd.concat([content_based_df, collaborative_based_df, hybrid_model_df])

    # Group by 'Title' and keep the row with the highest 'Rating' for each movie
    all_recommendations_df = all_recommendations_df.groupby('Title', as_index=False)['Rating'].max()

    # Sort by rating in descending order
    all_recommendations_df = all_recommendations_df.sort_values(by='Rating', ascending=False)

    return all_recommendations_df


In [ ]:
all_recommendations_df = get_recommendations(1, 'It Takes Two (1995)')
all_recommendations_df

,Title,Rating
10,Good Will Hunting (1997),4.255159
7,"Face in the Crowd, A (1957)",4.255159
18,October Sky (1999),4.255159
15,It's a Wonderful Life (1946),4.255159
13,It Happened One Night (1934),4.255159
12,Inherit the Wind (1960),4.255159
8,For All Mankind (1989),4.255159
19,"Shawshank Redemption, The (1994)",4.255159
5,"Conformist, The (Il Conformista) (1970)",4.255159
4,Bringing Up Baby (1938),4.255159


In [ ]:
import math

In [ ]:
def get_metrics():
    mae = 0
    rmse = 0
    se = 0
    mse = 0
    count = 0

    for user_id in users['User_ID']:
        for movie in df2['Title']:
            recommendations = get_recommendations(1, 'It Takes Two (1995)')
            ratings = recommendations['Rating'].values
            for rating in ratings:
                mae  = mae + abs(5-rating)
                se = se + ((5-rating)*(5-rating))
                count = count+1

    mse = se/count
    rmse = math.sqrt(se/count)
    mae = mae/count
    return mae, mse, rmse

In [ ]:
mae, mse, rmse = get_metrics()

In [ ]:
print(f'MAE of ensemble recommender system: {mae}')
print(f'MSE of ensemble recommender system: {mse}')
print(f'RMSE of ensemble recommender system: {rmse}')

MAE of ensemble recommender system: 0.8483287096674975
MSE of ensemble recommender system: 0.7789308878172087
RMSE of ensemble recommender system: 0.8825706135019501
